# Implementation of Basic Homebrew CNN

In [68]:
# General imports 
import sys
import os 
sys.path.insert(1, os.path.join(os.pardir, 'src'))
from itertools import product

# Data imports
import torch
import plotly.express as px
import numpy as np
import mlflow
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

# Homebrew imports 
import model
from utils import one_hot_encode_index
from optimizers import Adam
from activations import Softmax, ReLU
from layers import Dropout, LinearLayer, ConvolutionLayer, PoolingLayer, FlattenLayer
from loss import CategoricalCrossEntropyLoss

## TESTING 
import importlib
importlib.reload(model)
##

<module 'model' from '..\\src\\model.py'>

## Data loaders

In [69]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(32),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                                      ])

test_transforms = transforms.Compose([transforms.Resize(33),
                                      transforms.CenterCrop(32),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                                    ])

# setting up data loaders
data_dir = os.path.join(os.pardir, 'data', 'Plant_leave_diseases_32')

train_data = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_transforms)
test_data = datasets.ImageFolder(os.path.join(data_dir, 'validation'), transform=test_transforms)

### Train config

In [70]:
# Configs 
config = {
    'max_epochs': 100,
    'learning_rate': 0.003,
    'resolution': 32,
    'name': 'CNN_LeNet_inspired_homebrew'
}

## Model

In [71]:
mdl = model.Model(Adam(learning_rate=config['learning_rate']),
                      CategoricalCrossEntropyLoss())

# Config early stop 
mdl.add_early_stop(5)

mdl.set_save_config(model_name=config['name'], save_path=os.path.join('models'))

# Defining architecture 

mdl.set_sequence([
                    ConvolutionLayer(3, 18, 5),
                    ReLU(),
                    PoolingLayer(18, 2),
                    ConvolutionLayer(18, 28, 5),
                    ReLU(),
                    PoolingLayer(28, 2),
                    FlattenLayer(),
                    LinearLayer(700, 120),
                    ReLU(),
                    LinearLayer(120, 39),
                    Softmax()
                ])
print(mdl)

Model Architecture: 
+------------------+-----------+------------+
|      Layer       | Trainable | Parameters |
+------------------+-----------+------------+
| ConvolutionLayer |    True   |    1350    |
|       ReLU       |   False   |     0      |
|   PoolingLayer   |   False   |     0      |
| ConvolutionLayer |    True   |   12600    |
|       ReLU       |   False   |     0      |
|   PoolingLayer   |   False   |     0      |
|   FlattenLayer   |   False   |     0      |
|   LinearLayer    |    True   |   84120    |
|       ReLU       |   False   |     0      |
|   LinearLayer    |    True   |    4719    |
|     Softmax      |   False   |     0      |
+------------------+-----------+------------+
Total: 102,789


In [72]:
mlflow.set_experiment("Plant Leaf Disease")

train_loader = torch.utils.data.DataLoader(train_data, batch_size=256, shuffle=True)
validation_loader = torch.utils.data.DataLoader(test_data, batch_size=256, shuffle=True)


with mlflow.start_run():
    mlflow.log_param('framework', 'homebrew')
    mlflow.log_param('data_split', '90/10')
    mlflow.log_param('type', 'Basic_CNN_LeNet')
    mlflow.log_param('trainable_parameters', mdl.get_parameters_())
    mlflow.log_params(config)
    mdl.train_with_loader(train_loader, epochs=config['max_epochs'], validation_loader=validation_loader, cls_count=39, log_freq=5)

=== Epoch: 1 ===
Step: 0/217, accuracy0.023, loss3.893, learning rate 0.0030000 
Step: 5/217, accuracy0.121, loss9.914, learning rate 0.0030000 
Step: 10/217, accuracy0.074, loss3.628, learning rate 0.0030000 
Step: 15/217, accuracy0.051, loss3.643, learning rate 0.0030000 
Step: 20/217, accuracy0.094, loss3.591, learning rate 0.0030000 
Step: 25/217, accuracy0.078, loss3.601, learning rate 0.0030000 
Step: 30/217, accuracy0.090, loss3.651, learning rate 0.0030000 
Step: 35/217, accuracy0.094, loss3.515, learning rate 0.0030000 
Step: 40/217, accuracy0.105, loss3.495, learning rate 0.0030000 
Step: 45/217, accuracy0.043, loss3.564, learning rate 0.0030000 
Step: 50/217, accuracy0.082, loss3.493, learning rate 0.0030000 
Step: 55/217, accuracy0.105, loss3.507, learning rate 0.0030000 
Step: 60/217, accuracy0.098, loss3.504, learning rate 0.0030000 
Step: 65/217, accuracy0.070, loss3.486, learning rate 0.0030000 
Step: 70/217, accuracy0.109, loss3.483, learning rate 0.0030000 
Step: 75/2